# Analyze hepatocyte nuclei

Hepatocytes are often polyploidy and sometimes also binucleated. It seems 4n, 8n, and binucleation have some biological implications. It would be amazing to detect these phenotypes based on CellProfiiler features, and investigate whether some compounds perturb this phenotype.

In [19]:
import polars as pl
import sqlite3
import multiprocessing as mp

In [3]:
def get_feat_names(conn, table_name):
    # This function returns only the non-brightfield features
    query = f"PRAGMA table_info({table_name})"
    columns_info = conn.execute(query).fetchall()
    column_names = [info[1] for info in columns_info]

    return [i for i in column_names if "Brightfield" not in i]

def read_chunk(input_file, query, start_row, chunk_size):
    conn = sqlite3.connect(input_file)
    query_with_limit = f"{query} LIMIT {chunk_size} OFFSET {start_row}"
    df_chunk = pl.read_database(query_with_limit, conn)
    conn.close()
    return df_chunk

In [ ]:
# Get single-cell profiles from sqlite

In [4]:
prof_path = "/Users/jewald/Downloads/plate_41002880.sqlite"



In [7]:
conn = sqlite3.connect(prof_path)
# Create a cursor object
cursor = conn.cursor()

# Query to get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all table names
tables = cursor.fetchall()

# Print the table names
for table in tables:
    print(table[0])

# Close the connection
conn.close()

Image
Cells
Cytoplasm
Nuclei


In [9]:
conn = sqlite3.connect(prof_path)

# Get columns of interest
feats = get_feat_names(conn, "Nuclei")

In [12]:
query_total_rows = f"SELECT COUNT(*) FROM Nuclei"
total_rows = conn.execute(query_total_rows).fetchone()[0]
conn.close()

In [20]:
query = f"SELECT {', '.join(feats)} FROM Nuclei"
chunks = [(prof_path, query, start, 30000) for start in range(0, total_rows, 30000)]

# Use multiprocessing to read chunks in parallel
with mp.Pool(processes=4) as pool:
    df_chunks = pool.starmap(read_chunk, chunks)

# Concatenate all the DataFrames into a single DataFrame
full_df = pl.concat(df_chunks)

Process SpawnPoolWorker-2:
Process SpawnPoolWorker-3:
Process SpawnPoolWorker-4:
Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/Users/jewald/miniforge3/envs/axiom/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/jewald/miniforge3/envs/axiom/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jewald/miniforge3/envs/axiom/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/jewald/miniforge3/envs/axiom/lib/python3.9/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
Traceback (most recent call last):
AttributeError: Can't get attribute 'read_chunk' on <module '__main__' (built-in)>
  File "/Users/jewald/miniforge3/envs/axiom/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/jewald/miniforge3/envs/axiom/lib/python3.9/multiprocessing/p

KeyboardInterrupt: 

In [17]:
conn = sqlite3.connect(input_file)

# Get columns of interest
feats = get_feat_names(conn, "Nuclei")

# Get the total number of rows in the table
query_total_rows = f"SELECT COUNT(*) FROM Nuclei"
total_rows = conn.execute(query_total_rows).fetchone()[0]
conn.close()

# Define the query to read the table
query = f"SELECT {', '.join(feats)} FROM {table_name}"

# Calculate the number of chunks needed
chunks = [(prof_path, query, start, 30000) for start in range(0, total_rows, 30000)]

# Use multiprocessing to read chunks in parallel
with mp.Pool(processes=num_processes) as pool:
    df_chunks = pool.starmap(read_chunk, chunks)

# Concatenate all the DataFrames into a single DataFrame
full_df = pl.concat(df_chunks)

NameError: name 'input_file' is not defined